# E-Commerce Data Analysis: Uncovering Customer Behavior 🛍️

This notebook dives into the **Online Retail dataset** from the UCI Machine Learning Repository. This is a transactional dataset containing all transactions occurring between 12/01/2010 and 12/09/2011 for a UK-based online retail company.

Our objective is to perform a visualization to understand sales patterns, identify top customers and products, and segment customers based on their purchasing behavior. We will use **Seaborn**, **Matplotlib**, **Plotly**, and **Scikit-learn** to create insightful visualizations and models.

---
## 1. Setup and Data Loading

First, we import the necessary libraries. Note that since the online dataset is an Excel file, we will need the `openpyxl` engine for `pandas` to read it.

In [1]:
# essential libraries
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px
import datetime as dt
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA

# plot style for better aesthetics
sns.set_theme(style="whitegrid", palette="mako")
plt.rcParams['figure.figsize'] = (12, 7)

#the dataset from a URL
url = 'https://archive.ics.uci.edu/ml/machine-learning-databases/00352/Online%20Retail.xlsx'
df = pd.read_excel(url)

print("Dataset loaded successfully!")
df.head()

Dataset loaded successfully!


,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom


---
## 2. Data Cleaning and Preprocessing

Transactional datasets are often messy. We need to handle missing values, incorrect entries (like negative quantities, which represent returns), and format the data types correctly for analysis.

In [2]:
# missing values
print("Missing values per column:")
print(df.isnull().sum())

# --- Data Cleaning Steps ---
# 1. Dropping rows where CustomerID is null, as they are not attributable to a specific customer.
df.dropna(axis=0, subset=['CustomerID'], inplace=True)

# 2. Removing returns (transactions with negative quantity)
df = df[df['Quantity'] > 0]

# 3. Removing entries with zero unit price
df = df[df['UnitPrice'] > 0]

# 4. Converting CustomerID to integer type
df['CustomerID'] = df['CustomerID'].astype('int64')

print("\nData cleaning complete!")
print(f"Dataset now has {df.shape[0]} rows.")

Missing values per column:
InvoiceNo           0
StockCode           0
Description      1454
Quantity            0
InvoiceDate         0
UnitPrice           0
CustomerID     135080
Country             0
dtype: int64

Data cleaning complete!
Dataset now has 397884 rows.


### Feature Engineering

To facilitate our analysis, we will create new columns from the existing data, such as `TotalPrice` and various time-based features from `InvoiceDate`.

In [3]:
# Creating TotalPrice column
df['TotalPrice'] = df['Quantity'] * df['UnitPrice']

# Converting InvoiceDate to datetime objects and extract time features
df['InvoiceDate'] = pd.to_datetime(df['InvoiceDate'])
df['YearMonth'] = df['InvoiceDate'].map(lambda date: 100*date.year + date.month)
df['Month'] = df['InvoiceDate'].dt.month_name()
df['Day'] = df['InvoiceDate'].dt.day_name()
df['Hour'] = df['InvoiceDate'].dt.hour

print("New features created:")
df[['InvoiceDate', 'TotalPrice', 'YearMonth', 'Month', 'Day', 'Hour']].head()

New features created:


,InvoiceDate,TotalPrice,YearMonth,Month,Day,Hour
0,2010-12-01 08:26:00,15.30,201012,December,Wednesday,8
1,2010-12-01 08:26:00,20.34,201012,December,Wednesday,8
2,2010-12-01 08:26:00,22.00,201012,December,Wednesday,8
3,2010-12-01 08:26:00,20.34,201012,December,Wednesday,8
4,2010-12-01 08:26:00,20.34,201012,December,Wednesday,8


---
## 3. 2D Visualizations: Exploring Sales and Geographic Trends

With our data cleaned, we can now create visualizations to understand the business performance.

### Monthly Sales Trend

Let's start by looking at the total revenue generated each month.

In [10]:
# Grouping data by YearMonth and sum TotalPrice
monthly_sales = df.groupby('YearMonth')['TotalPrice'].sum().reset_index()

# Plotting monthly sales
fig = px.line(monthly_sales,
              x='YearMonth',
              y='TotalPrice',
              title='Total Monthly Sales (Dec 2010 - Nov 2011)',
              labels={'TotalPrice': 'Total Sales', 'YearMonth': 'Month'},
              markers=True)
fig.update_layout(xaxis = dict(tickmode = 'array', tickvals = monthly_sales['YearMonth'], ticktext = monthly_sales['YearMonth']))
fig.show()

#### Insight

The line chart clearly shows a strong upward trend in sales throughout the year, with a significant spike in **November 2011**. This is likely due to holiday season shopping, indicating a crucial period for the business.

### Sales by Day of the Week and Hour

Understanding when customers are most active can help in planning marketing campaigns and server maintenance.

In [11]:
# Sales by Day of Week
day_order = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
day_sales = df['Day'].value_counts().reindex(day_order).reset_index()
day_sales.columns = ['Day', 'Number of Orders']

fig_day = px.bar(day_sales,
                 x='Day',
                 y='Number of Orders',
                 title='Number of Orders by Day of the Week',
                 category_orders={"Day": day_order})
fig_day.show()

# Sales by Hour
hour_sales = df['Hour'].value_counts().sort_index().reset_index()
hour_sales.columns = ['Hour', 'Number of Orders']

fig_hour = px.bar(hour_sales,
                  x='Hour',
                  y='Number of Orders',
                  title='Number of Orders by Hour of the Day')
fig_hour.show()

#### Insight

- **Day of the Week**: The business appears to be most active on weekdays, with **Thursday** being the peak day. There are no sales recorded on Saturdays, which might indicate that the business doesn't operate on that day or there's an issue with the data for Saturdays.
- **Hour of the Day**: The busiest hours are from **12 PM to 3 PM**, which is the middle of the business day. This suggests that many customers might be shopping during their lunch breaks.

### Geographic Analysis: Sales by Country

Let's see where the customers are located. We will use an interactive plot from Plotly.

In [6]:
# Grouping by country and sum the total price
country_sales = df.groupby('Country')['TotalPrice'].sum().sort_values(ascending=False).reset_index()

# Creating an interactive bar chart for the top 15 countries
fig = px.bar(country_sales.head(15),
             x='Country',
             y='TotalPrice',
             title='Top 15 Countries by Total Sales',
             labels={'TotalPrice': 'Total Sales', 'Country': 'Country'})
fig.show()

#### Insight

Unsurprisingly, the vast majority of sales come from the **United Kingdom**, as it's a UK-based retailer. However, there are also significant international sales from countries like the **Netherlands, EIRE (Ireland), Germany, and France**.

---
## 4. Customer Segmentation with RFM Analysis

To understand our customers better, we'll segment them using **RFM (Recency, Frequency, Monetary)** analysis. This is a powerful marketing technique used to identify a company's best customers.
- **Recency**: How recently a customer made a purchase.
- **Frequency**: How often they make purchases.
- **Monetary**: How much money they spend.

In [7]:
# For Recency, we need to find the most recent purchase date for each customer
# We'll set a snapshot date, which is one day after the last transaction date in the dataset
snapshot_date = df['InvoiceDate'].max() + dt.timedelta(days=1)

# Aggregating data on a customer level
rfm = df.groupby('CustomerID').agg({
    'InvoiceDate': lambda date: (snapshot_date - date.max()).days, # Recency
    'InvoiceNo': 'nunique', # Frequency
    'TotalPrice': 'sum' # Monetary
}).rename(columns={'InvoiceDate': 'Recency', 'InvoiceNo': 'Frequency', 'TotalPrice': 'MonetaryValue'})

print("RFM Table created:")
rfm.head()

RFM Table created:


,Recency,Frequency,MonetaryValue
CustomerID,,,
12346,326,1,77183.60
12347,2,7,4310.00
12348,75,4,1797.24
12349,19,1,1757.55
12350,310,1,334.40


### 3D Visualization of RFM Segments

Let's visualize the RFM values in a 3D scatter plot to see if we can spot any natural clusters.

In [8]:
# Interactive 3D Scatter Plot of RFM
fig_3d = px.scatter_3d(rfm,
                       x='Recency',
                       y='Frequency',
                       z='MonetaryValue',
                       title='3D View of Customer Segments (RFM)',
                       color='MonetaryValue',
                       opacity=0.7)

fig_3d.update_traces(marker=dict(size=5))
fig_3d.show()

#### Insight

The 3D plot shows that most customers are clustered in the region of **low recency, low frequency, and low monetary value**. However, there are clear outliers who have high frequency and monetary value. These are likely our most valuable customers. The distribution is heavily skewed, which we should address before clustering.

---
## 5. Higher-Dimensionality Visualization with K-Means & PCA

We can formalize the segmentation by using the K-Means clustering algorithm. Because the RFM values are skewed, we'll first apply a log transformation and then scale the data. Finally, we'll use PCA to visualize the resulting clusters in 2D.

In [12]:
# 1. Handling skewness with log transformation and scale the data
rfm_log = np.log1p(rfm) # Use log1p to handle potential zero values
scaler = StandardScaler()
rfm_scaled = scaler.fit_transform(rfm_log)

# 2. Using K-Means to create clusters
# choosing 3 clusters for simplicity (e.g., 'Low Value', 'Mid Value', 'High Value')
kmeans = KMeans(n_clusters=3, random_state=42, n_init=10)
kmeans.fit(rfm_scaled)
rfm['Cluster'] = kmeans.labels_

# 3. Applying PCA to reduce the 3 RFM dimensions to 2 for visualization
pca = PCA(n_components=2)
principal_components = pca.fit_transform(rfm_scaled)
pca_df = pd.DataFrame(data=principal_components, columns=['PC1', 'PC2'])
pca_df['Cluster'] = rfm['Cluster'].values

# 4. Plotting the results
fig_pca = px.scatter(pca_df,
                     x='PC1',
                     y='PC2',
                     color='Cluster',
                     title='Customer Segments Visualized with PCA',
                     labels={'PC1': f'Principal Component 1 ({pca.explained_variance_ratio_[0]*100:.2f}% variance)',
                             'PC2': f'Principal Component 2 ({pca.explained_variance_ratio_[1]*100:.2f}% variance)'})
fig_pca.show()

#### Insight

This PCA plot successfully visualizes our customer segments, which were created in 3D (RFM space), in a simple 2D chart. The two principal components capture over 90% of the information from the original three RFM features.

We can clearly see three distinct groups of customers. We could now analyze the RFM characteristics of each cluster to define them. For example, one cluster would contain our **'Champions'** (high frequency/monetary, low recency), while another might contain **'At-Risk'** customers (high recency, previously high frequency/monetary).

---
## 6. Final Conclusion

This deep dive into the e-commerce dataset has provided valuable insights into the business's operations and customer base:

1.  **Sales are Seasonal**: There is a clear and strong seasonal peak in November, likely tied to holiday shopping, representing a critical revenue period.
2.  **Peak Shopping Times**: Customers are most active during weekday lunch hours (12 PM - 3 PM), offering a prime window for targeted promotions.
3.  **Geographic Concentration**: While the business has an international reach, the UK market is overwhelmingly dominant and should remain the primary focus.
4.  **Customer Segmentation is Key**: Customers are not monolithic. RFM analysis combined with K-Means and PCA has shown that we can identify distinct customer groups. This allows for tailored marketing strategies to retain high-value customers and re-engage those at risk of churning.

By leveraging a variety of visualization techniques, from simple 2D charts to more advanced multi-dimensional analysis, we have transformed raw transactional data into actionable business intelligence.